## 打印可视化界面

In [ ]:
from onekey_algo.datasets.image_loader import default_loader
from onekey_algo.custom.components.comp2 import show_cam_on_image
import torch
import os
import random

from onekey_algo.custom.components.comp2 import extract, init_from_model, init_from_onekey
from onekey_algo.utils.MultiProcess import MultiProcess
import os
import matplotlib
matplotlib.use('Agg')
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import monai
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd

samples = glob(r'F:\20230915-BianYun\patches/*/*.jpg')
model_dir = r'F:\20230915-BianYun\models\resnet18'
random.shuffle(samples)
def viz_sample(samples, thread_id):
    model, transformer, device = init_from_onekey(os.path.join(model_dir, 'viz'))
    for n, m in model.named_modules():
        print('Feature name:', n, "|| Module:", m)
    target_layer = "layer4.2.conv3"
    target_layer = r'layer4.1.conv2'
    gradcam = monai.visualize.GradCAM(nn_module=model, target_layers=target_layer)

    random.shuffle(samples)
    viz_dir = os.path.join(model_dir, 'Grad-CAM')
    os.makedirs(viz_dir, exist_ok=True)
    for sample in samples:
        if not os.path.exists(sample):
            continue
        img = default_loader(sample)
        sample_ = transformer(img)
        sample_  = sample_.view(1, *sample_.size()).to(device)
        res_cam = gradcam(x=sample_, class_idx=None)
        fig, axes = plt.subplots(1, 2, figsize=(20, 10), facecolor='white')
    #     axes[0].imshow(-res_cam[0][0].cpu(), cmap='jet')
        axes[0].imshow(img.resize(sample_.size()[2:]))
        axes[0].axis('off')
    #     plt.savefig(f"viz/{os.path.basename(sample).replace('.png', '_se.png')}", bbox_inches = 'tight')
    #     plt.show()
    #     plt.figure(figsize=(10, 10))
    #     plt.axis('off')
        imshow = axes[1].imshow(show_cam_on_image(img.resize(sample_.size()[2:]), -res_cam[0][0].cpu(), use_rgb=True, reverse=False), 
                                cmap='jet')
        axes[1].axis('off')
        cax = fig.add_axes([0.92, 0.15, 0.02, axes[1].get_position().height]) 
        plt.colorbar(imshow, cax=cax)
        plt.savefig(f'{viz_dir}/{os.path.basename(sample)}', bbox_inches = 'tight')
        plt.close()
viz_sample(samples, thread_id=1)
# MultiProcess(func=viz_sample, samples=samples, num_process=1).run()

Feature name:  || Module: ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (re